<a href="https://colab.research.google.com/github/Mutasar/Membangun-Proyek-Machine-Learning/blob/main/Analisa_Sentimen_Ulasan_di_Tokopedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pada Bagian ke-1 ini saya akan melakukan labeling dan pemrosesan prediksi klasifikasi dengan machine learning serta analisa teks ulasan pada Tokopedia.

# 1. Import Library

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
import joblib
from google.colab import files

# Import dataset yang telah dilakukan text preprocessing

In [ ]:
data = pd.read_excel("../input/texttokped/tokped_text.xlsx", index_col=0)

data.head()

# Labeling

Pembagian data menjadi data sentimen berlabel positif dan negatif dengan angka 1 untuk positif dan angka 0 untuk negatif. Pengklasifikasian ini dilakukan pada ulasan yang memiliki rating 4 dan 5 sebagai sentimen positif dan rating 3 sampai 1 sebagai sentimen negatif.

In [ ]:
label = []
for index, row in data.iterrows():
    if row["rate"] == 5 or row["rate"] == 4:
        label.append(1)
    else:
        label.append(0)

data["label"] = label

data

In [ ]:
data["label"].value_counts()

# Menghitung Kata Dengan TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
Ulasan = data['Ulasan_clean']

In [ ]:
Ulasan.isnull().sum()

In [ ]:
Ulasan = Ulasan.fillna('tidak ada komentar')

In [ ]:
#untuk menghitung jumlah kata yang telah di steming
cv = CountVectorizer()
term_fit = cv.fit(Ulasan)

print (len(term_fit.vocabulary_))

In [ ]:
term_fit.vocabulary_ #mengurutkan berdasarkan urutab abjad kata

In [ ]:
#kolom pertama ini berarti jumlah dokumen
#kolom kedua berarti letak katanya
#kolom ketiga hasil dari tf

term_frequency_all = term_fit.transform(Ulasan)
print (term_frequency_all)

In [ ]:
ulasan_tf = Ulasan[1] #memanggil kata pada index ke 1
print (ulasan_tf)

In [ ]:
term_frequency = term_fit.transform([ulasan_tf]) #hanya menampilkan hasil document 1
print (term_frequency)

In [ ]:
dokumen = term_fit.transform(Ulasan) #hasil perhitungan tf idf dalam 1 doc
tfidf_transformer = TfidfTransformer().fit(dokumen)
print (tfidf_transformer.idf_)

tfidf=tfidf_transformer.transform(term_frequency)
print (tfidf) #hasil manual dengan sistem pyhton

# NLP

In [ ]:
data_label = data[["Nama_Produk", "Akun", "Ulasan_clean", "label"]]

In [ ]:
data_label["Ulasan_clean"] = data_label["Ulasan_clean"].fillna("tidak ada komentar")

In [ ]:
data_label.to_excel("data_label.xlsx")

In [ ]:
sentimen_data=pd.value_counts(data_label["label"], sort= True)
sentimen_data.plot(kind= 'bar', color= ["green", "red"])
plt.title('Bar chart')
plt.show()

Dapat dilihat bahwa isi ulasan produk lebih banyak pada label sentimen 1 atau ulasan dengan rating postitif ini berarti pelanggan yang menggunakan marketplace Tokopedia dan melakukan transaksi pembelian pada produk masker Kesehatan merasa puas bertansaksi di marketplace Tokopedia dan prosuk masker Kesehatan sehingga memberikan feedback atau ulasan komentar lebih banyak yang positif.

In [ ]:
from wordcloud import WordCloud

**Ulasan Negatif**

In [ ]:
train_s0 = data_label[data_label["label"] == 0]

In [ ]:
train_s0["Ulasan_clean"] = train_s0["Ulasan_clean"].fillna("tidak ada komentar")

In [ ]:
train_s0

In [ ]:
all_text_s0 = ' '.join(word for word in train_s0["Ulasan_clean"])
wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s0)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

Dari visualisasi diatas merupakan wordcloud kata yang paling banyak muncul pada isi ulasan yang memiliki label sentimen negatif. Kata yang paling sering muncul dan mengarah ke ulasan negatif membahas seputar : barang, harga, kotak, penyok, box, dus, tipis, putus, sobek, kualitas, karet, bolong dan sebagainya. Sehingga dari kata-kata ini bisa menjadi masukan untuk penjual dan marketplace Tokopedia untuk meningkatkan kualitas barang (produk masker kesehatan), harga, kualitas pengiriman atau pengemasan, serta kualitas produk masker Kesehatan yang paling banyak disebutkan pelanggan dalam hasil Analisa ulasan sentimen yang negatif.

**Ulasan Positif**

In [ ]:
train_s1 = data_label[data_label["label"] == 1]

In [ ]:
train_s1["Ulasan_clean"] = train_s1["Ulasan_clean"].fillna("tidak ada komentar")

In [ ]:
train_s1

In [ ]:
all_text_s1 = ' '.join(word for word in train_s1["Ulasan_clean"])
wordcloud = WordCloud(colormap='Blues', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s1)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Ulasan Positif")
plt.margins(x=0, y=0)
plt.show()

Dari visualisasi diatas merupakan wordcloud kata yang paling banyak muncul pada ulasan yang memiliki label sentimen positif. Kata yang paling sering muncul dan mengarah ke ulasan positif membahas seputar : barang, cepat, bagus, masker, aman, kualitas, sesuai, rapi, respon, aman, recommended, dan sebagainya. Sehingga dari kata-kata ini bisa menjadi masukan untuk penjual dan marketplace Tokopedia untuk menjaga kualitas atau meningkatkan kembali kualitas barang (produk masker kesehatan), kualitas yang sesuai dan aman, serta respon penjual paling banyak disebutkan pelanggan dalam hasil Analisa ulasan sentimen yang positif.

# Menyiapkan Data Train dan Test

Pada proses ini kami menggunakan library sklearn.model_selection dengan modul train_test _split untuk membagi data latih (X_train dan y_train) dan data uji (X_test dan y_test) dengan persentasi data latih 70% dan data uji 30% serta memilih label data yaitu yang merupakan variable independen dari data kami yaitu kolom label untuk dijadikan parameter klasifikasi prediksi.

In [ ]:
data_label['Ulasan_clean'] = data_label['Ulasan_clean'].fillna("tidak ada komentar")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_label['Ulasan_clean'], data_label['label'],
                                                    test_size=0.1, stratify=data_label['label'], random_state=30)

# TF-IDF

Pada proses ini kami menggunakan pembobotan TF-IDF(term frequency–inverse document) untuk menghitung manual dengan menggunakan python pembobotan kata dalam dokumen data ulasan.

In [ ]:
import numpy as np

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8')

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train = X_train.toarray()

In [ ]:
X_test = X_test.toarray()

# Machine Learning

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

#deklarasi metode cross validation
cv_method = RepeatedStratifiedKFold(n_splits=5,  n_repeats=3, random_state=999)
#tuning hyperparameter menggunakan gridsearch

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gscv_nb = GridSearchCV(estimator=nb,
                 param_grid=params_NB,
                 cv=cv_method,   # use any cross validation technique
                 verbose=1,
                 scoring='accuracy')

#Fitting ke Model
gscv_nb.fit(X_train,y_train)
#mendapatkan hyperparameters terbaik
gscv_nb.best_params_

In [ ]:
nb = GaussianNB(var_smoothing=1.0)

In [ ]:
nb.fit(X_train, y_train)

In [ ]:
y_pred_nb = nb.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print('--------------------- confusion matrix  ----------------------------')
print(confusion_matrix(y_test, y_pred_nb))
print('--------------------- classification report  ----------------------------')
print(classification_report(y_test, y_pred_nb))

Setelah dilakukan pembagian data latih dan data uji serta pembobotan tf-idf selanjutnya dapat dilakukan proses klasifikasi prediksi menggunakan model algoritma Naïve Bayes seperti proses yang ditunjukkan pada gambar 4. Didapatkan model algoritma Naïve Bayes dapat memberikan akurasi yang cukup baik sampari 88%.

> Dari hasil penelitian menggunakan Metode Algoritma Naïve Bayes untuk mengetahui sentimen ulasan pengguna dengan klasifikasi 2 kelas positif dan negative dengan pendekatan NLP menghasilkan nilai akurasi sebesar 88%. Selain itu, didapatkan bahwa Analisa Sentimen pada ulasan marketplace Tokopedia pada produk masker kesehatan menunjukan lebih banyak pada ulasan yang positif. Ini berarti pelayanan dan produk masker Kesehatan yang disediakan di marketplace Tokopedia sudah cukup baik.


> Dari hasil Analisis diatas dapat disimpulkan hasil scraping yang kami dapat dari produk pencarian masker kesehatan pada Tokopedia menampilkan data yang menunjukan ulasan positif lebih dominan daripada hasil ulasan negatif dan untuk ulasan negatif kata yang paling sering muncul adalah seputar kualitas produk yang tipis,mudah putus, sobek atau bolong, kualitas karetnya dan  pada pengemasan yaitu kotak/dus penyok dan untuk analisa positif kata yang paling sering muncul adalah seputar kualitas produk yang sesuai dan rapi, pengiriman yang aman dan cepat dan respon penjual.